In [147]:
from lxml import html
from json import dump, loads
from requests import get
import json
import re
from dateutil import parser as dateparser_to_html
import urllib3
from time import sleep

In [3]:
asin = 'B07CXMQ6X6'

In [4]:
amazon_url = 'https://www.amazon.com/product-reviews/' + asin + '/ref=cm_cr-arp_d_paging_btm_next_1?pageNumber=1'

In [5]:
urllib3.disable_warnings()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}
response = get(amazon_url, headers=headers, verify=False, timeout=30)

In [10]:
cleaned_response = response.text.replace('\x00','')
parser_to_html = html.fromstring(cleaned_response)

In [55]:
number_review = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[1]/div[2]/div/div/div[2]/div/span//text()'))

In [26]:
total_ratings_line  = parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div')

In [44]:
''.join(total_ratings_line[0].xpath('//div[1]/div[3]/span/a/span/text()'))

'3.5 out of 5 starsVerified PurchaseVerified PurchaseVerified PurchaseVerified PurchaseVerified PurchaseVerified PurchaseVerified PurchaseVerified PurchaseVerified PurchaseVerified Purchase'

In [85]:
total_ratings  = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[1]/div[3]/span/a/span/text()')).split()[0]

In [86]:
total_ratings

'3.5'

In [87]:
page_number = 2

In [88]:
amazon_url = 'https://www.amazon.com/product-reviews/'+ asin + '/ref=cm_cr_arp_d-paging_btm_next_' + str(page_number) + '?pageNumber=' + str(page_number)
urllib3.disable_warnings()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}
response = get(amazon_url, headers=headers, verify=False, timeout=30)
cleaned_response = response.text.replace('\x00','')
parser_to_html = html.fromstring(cleaned_response)

In [100]:
parser_to_html

<Element html at 0x116cdae58>

In [101]:
reviews = parser_to_html.xpath('//div[@data-hook="review"]')

In [102]:
reviews

[<Element div at 0x116cd4958>,
 <Element div at 0x116d3d318>,
 <Element div at 0x116d3d6d8>,
 <Element div at 0x116d3d8b8>,
 <Element div at 0x116d3d9f8>,
 <Element div at 0x116d3da48>,
 <Element div at 0x116d3da98>,
 <Element div at 0x116d3dae8>,
 <Element div at 0x116d3db38>,
 <Element div at 0x116d3db88>]

In [106]:
reviews[0].xpath('//text()')

['\n',
 'var aPageStart = (new Date()).getTime();',
 '\n',
 'var ue_t0=ue_t0||+new Date();',
 '\n',
 '\nwindow.ue_ihb = (window.ue_ihb || window.ueinit || 0) + 1;\nif (window.ue_ihb === 1) {\n\nvar ue_csm = window,\n    ue_hob = +new Date();\n(function(d){var e=d.ue=d.ue||{},f=Date.now||function(){return+new Date};e.d=function(b){return f()-(b?0:d.ue_t0)};e.stub=function(b,a){if(!b[a]){var c=[];b[a]=function(){c.push([c.slice.call(arguments),e.d(),d.ue_id])};b[a].replay=function(b){for(var a;a=c.shift();)b(a[0],a[1],a[2])};b[a].isStub=1}};e.exec=function(b,a){return function(){if(1==window.ueinit)try{return b.apply(this,arguments)}catch(c){ueLogError(c,{attribution:a||"undefined",logLevel:"WARN"})}}}})(ue_csm);\n\n\n    var ue_err_chan = \'jserr-rw\';\n(function(d,e){function h(f,b){if(!(a.ec>a.mxe)&&f){a.ter.push(f);b=b||{};var c=f.logLevel||b.logLevel;c&&c!==k&&c!==m&&c!==n&&c!==p||a.ec++;c&&c!=k||a.ecf++;b.pageURL=""+(e.location?e.location.href:"");b.logLevel=c;b.attribution=f.attri

In [116]:
review_total_pages = []
review_dict = {}
for review in reviews:
    raw_review_author = review.xpath('.//span[contains(@class,"profile-name")]//text()')
    raw_review_rating = review.xpath('.//i[@data-hook="review-star-rating"]//text()')
    raw_review_header = review.xpath('.//a[@data-hook="review-title"]//text()')
    raw_review_posted_date = review.xpath('.//span[@data-hook="review-date"]//text()')
    raw_review_text1 = review.xpath('.//span[@data-hook="review-body"]//text()')
    raw_review_text2 = review.xpath('.//div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview')
    raw_review_text3 = review.xpath('.//div[contains(@id,"dpReviews")]/div/text()')
    
    author = ' '.join(' '.join(raw_review_author).split())
    review_rating = ''.join(raw_review_rating).replace('out of 5 stars', '')
    review_header = ' '.join(' '.join(raw_review_header).split())
    try:
        review_posted_date = dateparser_to_html.parse(''.join(raw_review_posted_date)).strftime('%d %b %Y')
    except:
        review_posted_date = None
    review_text = ' '.join(' '.join(raw_review_text1).split())

    # Grabbing hidden comments if present
    if raw_review_text2:
        json_loaded_review_data = loads(raw_review_text2[0])
        json_loaded_review_data_text = json_loaded_review_data['rest']
        cleaned_json_loaded_review_data_text = re.sub('<.*?>', '', json_loaded_review_data_text)
        full_review_text = review_text + cleaned_json_loaded_review_data_text
    else:
        full_review_text = review_text
    if not raw_review_text1:
        full_review_text = ' '.join(' '.join(raw_review_text3).split())
    
    review_dict = {
        'review_text': full_review_text,
        'review_posted_date': review_posted_date,
        'review_header': review_header,
        'review_rating': review_rating,
        'review_author': author
    }
    review_total_pages.append(review_dict)
print(review_total_pages)

[{'review_text': 'Guys...I’m OBSESSED with these!!! I buy a bit smaller than I usually would. Which lets be honest, makes me feel even better! I’d typically buy a large/short but get medium/short for these. I own two pairs so far and plan to get them in every color. They fit amazing. They’re comfy. And I look GREAT in them! I’m not a super fan of the jegging look, these look and feel like real jeans!', 'review_posted_date': '06 May 2019', 'review_header': 'Buy them. Buy them NOW!', 'review_rating': '5.0 ', 'review_author': 'Bellands'}, {'review_text': 'Absolutely love these jeans. I have a few pairs I different colors. I would originally give this style 5 ⭐️ BUT, they do not last long. Every pair I’ve purchased has never lasted longer than 6 months before the seams start to unravel.', 'review_posted_date': '28 Apr 2019', 'review_header': 'Short term wear', 'review_rating': '4.0 ', 'review_author': 'TEC'}, {'review_text': 'Hard pass on this pair. They look and feel like maternity pants.

In [115]:
for review in reviews:
    raw_review_rating = review.xpath('.//i[@data-hook="review-star-rating"]//text()')
    review_rating = ''.join(raw_review_rating).replace('out of 5 stars', '')
    print(review_rating)

5.0 
4.0 
1.0 
2.0 
1.0 
3.0 
1.0 
5.0 
5.0 
2.0 


In [113]:
review_total_pages

[{'review_text': 'Guys...I’m OBSESSED with these!!! I buy a bit smaller than I usually would. Which lets be honest, makes me feel even better! I’d typically buy a large/short but get medium/short for these. I own two pairs so far and plan to get them in every color. They fit amazing. They’re comfy. And I look GREAT in them! I’m not a super fan of the jegging look, these look and feel like real jeans!',
  'review_posted_date': '06 May 2019',
  'review_header': 'Buy them. Buy them NOW!',
  'review_rating': '',
  'review_author': 'Bellands'},
 {'review_text': 'Absolutely love these jeans. I have a few pairs I different colors. I would originally give this style 5 ⭐️ BUT, they do not last long. Every pair I’ve purchased has never lasted longer than 6 months before the seams start to unravel.',
  'review_posted_date': '28 Apr 2019',
  'review_header': 'Short term wear',
  'review_rating': '',
  'review_author': 'TEC'},
 {'review_text': 'Hard pass on this pair. They look and feel like matern

In [46]:
import re

In [53]:
txt = 'https://www.amazon.com/Daily-Ritual-Womens-Denim-Jegging/dp/B07CZ35NPV/ref=lp_19151634011_1_4?s=apparel&ie=UTF8&qid=1557417259&sr=1-4&nodeID=19151634011&psd=1'
txt.split('/')[txt.split('/').index('dp')+1]

'B07CZ35NPV'

In [54]:
txt.split('/')

['https:',
 '',
 'www.amazon.com',
 'Daily-Ritual-Womens-Denim-Jegging',
 'dp',
 'B07CZ35NPV',
 'ref=lp_19151634011_1_4?s=apparel&ie=UTF8&qid=1557417259&sr=1-4&nodeID=19151634011&psd=1']

In [117]:
def extract_asin(url):
    product_page_url = url
    url_parameters = product_page_url.split('/')
    if 'www.amazon.com' not in url_parameters:
        print("Please provide proper URL")
    else:
        return(url_parameters[url_parameters.index('dp')+1])

In [142]:
def get_header(asin):
    amazon_url = 'https://www.amazon.com/product-reviews/' + asin + '/ref=cm_cr-arp_d_paging_btm_next_1?pageNumber=1'
    try:
        urllib3.disable_warnings()
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
        }
        response = get(amazon_url, headers=headers, verify=False, timeout=30)
        cleaned_response = response.text.replace('\x00','')
        parser_to_html = html.fromstring(cleaned_response)

        number_reviews = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[1]/div[2]/div/div/div[2]/div/span//text() '))
        product_price = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[2]/div/div/div[2]/div[4]/span/span[3]/text()'))
        product_name = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[2]/div/div/div[2]/div[1]/h1/a/text()'))
        # Extract ratings out of 5 stars from the text line
        product_rating  = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[1]/div[3]/span/a/span/text()')).split()[0]

        number_page_reviews = int(int(number_reviews)/10)

        if int(number_reviews) % 10 == 0:
            number_page_reviews += 0
        else:
            number_page_reviews +=  1
        
        return product_price, product_name, number_reviews, product_rating, number_page_reviews

    except Exception as e:
        return {"url":amazon_url, "error":e}

In [148]:
def get_all_reviews(asin):
    review_total_pages = []
    product_price, product_name, number_reviews, ratings_dict, stop_loop_for = get_header(asin)
    for page_number in range(1, stop_loop_for):
        print(page_number)
        amazon_url = 'https://www.amazon.com/product-reviews/' + asin + '/ref=cm_cr_arp_d-paging_btm_next_' + str(
            page_number) + '?pageNumber=' + str(page_number)
        try:
            urllib3.disable_warnings()
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
            }
            response = get(amazon_url, headers=headers, verify=False, timeout=30)

            #Removing the null bytes from the response
            cleaned_response = response.text.replace('\x00','')
            parser_to_html = html.fromstring(cleaned_response)
        except Exception as e:
            return {"url": amazon_url, "error": e}
        reviews = parser_to_html.xpath('//div[@data-hook="review"]')

        for review in reviews:
            raw_review_author = review.xpath('.//span[contains(@class,"profile-name")]//text()')
            raw_review_rating = review.xpath('.//i[@data-hook="review-star-rating"]//text()')
            raw_review_header = review.xpath('.//a[@data-hook="review-title"]//text()')
            raw_review_posted_date = review.xpath('.//span[@data-hook="review-date"]//text()')
            raw_review_text1 = review.xpath('.//span[@data-hook="review-body"]//text()')
            raw_review_text2 = review.xpath('.//div//span[@data-action="columnbalancing-showfullreview"]/@data-columnbalancing-showfullreview')
            raw_review_text3 = review.xpath('.//div[contains(@id,"dpReviews")]/div/text()')

            author = ' '.join(' '.join(raw_review_author).split())
            review_rating = ''.join(raw_review_rating).replace('out of 5 stars', '')
            review_header = ' '.join(' '.join(raw_review_header).split())
            try:
                review_posted_date = dateparser_to_html.parse(''.join(raw_review_posted_date)).strftime('%d %b %Y')
            except:
                review_posted_date = None
            review_text = ' '.join(' '.join(raw_review_text1).split())

            # Grabbing hidden comments if present
            if raw_review_text2:
                json_loaded_review_data = loads(raw_review_text2[0])
                json_loaded_review_data_text = json_loaded_review_data['rest']
                cleaned_json_loaded_review_data_text = re.sub('<.*?>', '', json_loaded_review_data_text)
                full_review_text = review_text + cleaned_json_loaded_review_data_text
            else:
                full_review_text = review_text
            if not raw_review_text1:
                full_review_text = ' '.join(' '.join(raw_review_text3).split())

            review_dict = {
                'review_text': full_review_text,
                'review_posted_date': review_posted_date,
                'review_header': review_header,
                'review_rating': review_rating,
                'review_author': author
            }
            review_total_pages.append(review_dict)
            sleep(5)

    data ={
        'product_name': product_name,
        'product_price': product_price,
        'number_reviews': number_reviews,
        'ratings': ratings_dict,
        'reviews': review_total_pages,
    }
    return data

In [121]:
asin = extract_asin('https://www.amazon.com/Daily-Ritual-Womens-Corduroy-5-Pocket/dp/B07BSXG3RK/ref=pd_sbs_641_4/133-6911766-6659409?_encoding=UTF8&pd_rd_i=B07BSXG3RK&pd_rd_r=0b4c561d-7290-11e9-88ae-891106fe0638&pd_rd_w=ADkIf&pd_rd_wg=wQkhc&pf_rd_p=588939de-d3f8-42f1-a3d8-d556eae5797d&pf_rd_r=1SN0TQ589E8SCPMPNN6T&refRID=1SN0TQ589E8SCPMPNN6T')

In [122]:
asin

'B07BSXG3RK'

In [151]:
get_header(asin)

('$11.68 - $29.00',
 "Daily Ritual Women's Corduroy 5-Pocket Skinny Pant",
 '30',
 '2.8',
 3)

In [149]:
reviews = get_all_reviews(asin)

1
2


In [150]:
reviews

{'product_name': "Daily Ritual Women's Corduroy 5-Pocket Skinny Pant",
 'product_price': '$11.68 - $29.00',
 'number_reviews': '30',
 'ratings': '2.8',
 'reviews': [{'review_text': 'These could use a little more contour or stretch. The waist stands off of my body and the pants slip down annoyingly and require a belt (cinched pretty tight) to keep from losing them completely. Bear in mind, I don\'t have a tiny waist. I was hoping for thicker fabric for warmth to wear with boots and a sweater. I would classify these as "summer weight" corduroy. Which seems like it shouldn\'t be a thing, but apparently is. I wear a size 8-10.',
   'review_posted_date': '10 Mar 2019',
   'review_header': 'Lightweight fabric, not contoured properly for curves',
   'review_rating': '3.0 ',
   'review_author': 'That Girl Said'},
  {'review_text': 'I loved the look of these corduroy leggings, but as you wear them throughout the day they tend to really, really stretch out. They fit good at the beginning when I 

In [141]:
amazon_url = 'https://www.amazon.com/product-reviews/' + asin + '/ref=cm_cr-arp_d_paging_btm_next_1?pageNumber=1'
urllib3.disable_warnings()
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'
}
response = get(amazon_url, headers=headers, verify=False, timeout=30)
cleaned_response = response.text.replace('\x00','')
parser_to_html = html.fromstring(cleaned_response)

number_reviews = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[1]/div[2]/div/div/div[2]/div/span//text() '))
product_price = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[2]/div/div/div[2]/div[4]/span/span[3]/text()'))
product_name = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[2]/div/div/div[2]/div[1]/h1/a/text()'))
# Extract ratings out of 5 stars from the text line
product_rating  = ''.join(parser_to_html.xpath('//*[@id="cm_cr-product_info"]/div/div[1]/div[3]/span/a/span/text()')).split()[0]

number_page_reviews = int(int(number_reviews)/10)

if number_reviews % 10 == 0:
    number_page_reviews += 0
else:
    number_page_reviews +=  1

print(product_price, product_name, number_reviews, product_rating, number_page_reviews)

TypeError: not all arguments converted during string formatting